# Finding the Minimum Simulation Steps of Spiking Dijkstra

In [ ]:
try:
    # Check if this cell is running in google colab
    import google.colab

    !pip install gitpython
    !pip install simsnn==1.0.2
    import os, shutil
    from git import Repo


    # Delete everything in the content (current) directory on google colab
    for root, dirs, files in os.walk("/content"):
        for name in files:
            os.remove(os.path.join(root, name))
        for name in dirs:
            shutil.rmtree(os.path.join(root, name))
        break

    # Clone git repo, change the branch and
    Repo.clone_from("https://github.com/adiehl96/SpikingDijkstra.git", "/content/SpikingDijkstraa")

    # Move the repo up by one level in the folder hierarchy
    for root, dirs, files in os.walk("/content/SpikingDijkstraa"):
        for name in dirs + files:
            shutil.move(os.path.join(root, name), os.path.join("/content", name))
        break
    os.rmdir("/content/SpikingDijkstraa")

except:
    print("This cell is only supposed to be executed on google colab, not on your local machine!")

In [ ]:
### This hack allows accessing spijking dijkstra from this folder during development.
if "SCRIPT_DIR" not in vars(): import sys; import os; SCRIPT_DIR=os.getcwd(); sys.path.append(os.path.dirname(SCRIPT_DIR)); sys.path.append(SCRIPT_DIR);
###


import numpy as np
import networkx as nx
from pprint import pprint

from spikingdijkstra.graphsupport import is_equal, get_test_graphs
from spikingdijkstra.algorithms import sssp


def get_min_steps(gd, n):
    g = nx.fast_gnp_random_graph(gd["n"], gd["p"], seed=gd["s"])
    for e0, e1 in gd["weights"]:
        g[e0][e1]["weight"] = gd["weights"][(e0, e1)]
    nxpaths = nx.single_source_dijkstra(g, gd["source"])[1]
    for steps in range(n):
        snnpaths = sssp(g, gd["source"], steps=steps)
        try:
            assert is_equal(g, nxpaths, snnpaths)
            gd["minsteps"] = steps
            break
        except AssertionError:
            continue
    assert "minsteps" in gd
    return gd

def test_get_min_steps(max_simulation_steps=100):
    gds = get_test_graphs(range(3, 12), np.linspace(0.5, 1, 2), 5)
    for gd in gds:
        get_min_steps(gd, max_simulation_steps)
    return gds

In [ ]:
### This hack allows accessing spijking dijkstra from this folder during development.
if "SCRIPT_DIR" not in vars(): import sys; import os; SCRIPT_DIR=os.getcwd(); sys.path.append(os.path.dirname(SCRIPT_DIR), SCRIPT_DIR);
###

minstepsdict = test_get_min_steps(max_simulation_steps=100)
pprint(minstepsdict)